In [2]:
import tensorflow as tf
import argparse
import os
from config import cfg
from easydict import EasyDict
from utils.kitti_loader import iterate_data
from model.group_pointcloud import FeatureNet
import numpy as np
from model.rpn import MiddleAndRPN

In [2]:
parser=EasyDict()
parser.i = 1
parser.tag = 'Test1'
parser.single_batch_size = 2
parser.lr =0.001
parser.al =1
parser.output_path = './prediction'
parser.v=False

dataset_dir = cfg.DATA_DIR
train_dir = os.path.join(cfg.DATA_DIR, 'training')
val_dir = os.path.join(cfg.DATA_DIR, 'validation')
log_dir = os.path.join('./log', parser.tag)
save_model_dir = os.path.join('./save_model', parser.tag)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(save_model_dir, exist_ok=True)

In [3]:
batches=iterate_data(train_dir,batch_size=2)
#tag,labels,vox_feature,vox_number,vox_coordinate,rgb,raw_lidar
batch=next(batches)
voxel_feature=batch[2]
voxel_coordinate=batch[4]

In [4]:
A= tf.convert_to_tensor(voxel_coordinate)

In [5]:
A=tf.Variable([1.,2.,3.])
A

<tf.Variable 'Variable:0' shape=(3,) dtype=float32, numpy=array([1., 2., 3.], dtype=float32)>

In [6]:
tf.slice(A,[0],[1])

<tf.Tensor: id=13, shape=(1,), dtype=float32, numpy=array([1.], dtype=float32)>

In [7]:
type(np.int32(voxel_coordinate.shape[1]))==np.int32

True

In [8]:
print("{}:{}".format("tag",batch[0].dtype))
print("{}:{}".format("labels",batch[1].dtype))
print("{}:{}".format("vox_feature",batch[2].dtype))
print("{}:{}".format("vox_number",batch[3].dtype))
print("{}:{}".format("vox_coordinate",batch[4].dtype))
print("{}:{}".format("rgb",batch[5].dtype))
print("{}:{}".format("raw_lidar",batch[6][0].dtype))

tag:<U6
labels:object
vox_feature:float32
vox_number:int64
vox_coordinate:int64
rgb:uint8
raw_lidar:float32


In [18]:
import os
import time
import sys
import glob
import tensorflow as tf
from itertools import count
import argparse
from config import cfg
from easydict import EasyDict
from utils.kitti_loader import iterate_data,sample_test_data
import numpy as np
from utils.utils import *
from model.model import RPN3D
#from train_hook import check_if_should_pause

args=EasyDict()
args.i = 1
args.tag = 'Test1'
args.single_batch_size = 2
args.lr =0.001
args.al =1
args.output_path = './prediction'
args.v=False

dataset_dir = cfg.DATA_DIR
train_dir = os.path.join(cfg.DATA_DIR, 'training')
val_dir = os.path.join(cfg.DATA_DIR, 'validation')
log_dir = os.path.join('./log', args.tag)
save_model_dir = os.path.join('./save_model', args.tag)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(save_model_dir, exist_ok=True)

def get_data(batch):
    labels = batch[1]
    # transfer to tensor is for avoid tracking
    voxel_feature = tf.cast(batch[2],dtype=tf.float32)
    vox_number = tf.cast(batch[3],dtype=tf.int32) # original is int64
    voxel_coordinate =tf.cast(batch[4],dtype=tf.int32) # original is int64
    rgb = batch[5]
    raw_lidar = batch[6]

    # get from labels
    pos_equal_one, neg_equal_one, targets = cal_rpn_target(labels, [cfg.FEATURE_HEIGHT,cfg.FEATURE_WIDTH], anchors,cls=cfg.DETECT_OBJ,coordinate="lidar")
    pos_equal_one_for_reg = np.concatenate([np.tile(pos_equal_one[..., [0]], 7), np.tile(pos_equal_one[..., [1]], 7)], axis=-1)
    pos_equal_one_sum = np.clip(np.sum(pos_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
    neg_equal_one_sum = np.clip(np.sum(neg_equal_one, axis=(1, 2, 3)).reshape(-1, 1, 1, 1), a_min=1, a_max=None)
    pos_equal_one = tf.cast(pos_equal_one,dtype=tf.float32)
    neg_equal_one = tf.cast(neg_equal_one,dtype=tf.float32)
    targets = tf.cast(targets,dtype=tf.float32)
    pos_equal_one_for_reg = tf.cast(pos_equal_one_for_reg,dtype=tf.float32)
    pos_equal_one_sum = tf.cast(pos_equal_one_sum,dtype=tf.float32)
    neg_equal_one_sum = tf.cast(neg_equal_one_sum,dtype=tf.float32)
    return voxel_feature,vox_number,voxel_coordinate,pos_equal_one,neg_equal_one,targets,pos_equal_one_for_reg,pos_equal_one_sum,neg_equal_one_sum

if __name__ == '__main__':
    #tf.config.experimental_run_functions_eagerly(True)
    train_summary_writer = tf.summary.create_file_writer('./summaries/train/')
    tf.random.set_seed(1)

    batch_size=args.single_batch_size
    max_epoch = 1

    # training only for bn and let it run in training mode but not inference mode
    model=RPN3D(cls=cfg.DETECT_OBJ,batch_size=batch_size, alpha=1.0,beta=10.0,training=False)
    # save
    #model.save_weights(save_model_dir+'save_model_1')
    batch_time = time.time()
    counter=0
    summary_interval = 1
    anchors = cal_anchors()
    for epoch in range(max_epoch):
        for idx, batch in enumerate(iterate_data(train_dir,shuffle=True,aug=True,batch_size=batch_size,multi_gpu_sum=1,is_testset=False)):

            if idx==1:
                break
            tag = batch[0]
            print(tag)
            # get the data
            voxel_feature,vox_number,voxel_coordinate,pos_equal_one,neg_equal_one,targets,pos_equal_one_for_reg,pos_equal_one_sum,neg_equal_one_sum = get_data(batch)
            counter += 1
            start_time = time.time()

            if counter % summary_interval == 0:
                is_summary = True
            else:
                is_summary = False

            with train_summary_writer.as_default():
                ret=model.train_step(voxel_feature=voxel_feature,vox_number=vox_number,voxel_coordinate=voxel_coordinate,pos_equal_one=pos_equal_one,neg_equal_one=neg_equal_one,targets=targets,pos_equal_one_for_reg=pos_equal_one_for_reg,pos_equal_one_sum=pos_equal_one_sum,neg_equal_one_sum=neg_equal_one_sum,is_summary=is_summary)
            batch_time = time.time() - batch_time
            forward_time = time.time() - start_time

            print('train: {} @ epoch:{}/{} loss: {:.4f} reg_loss: {:.4f} cls_loss: {:.4f} cls_pos_loss: {:.4f} cls_neg_loss: {:.4f} forward time: {:.4f} batch time: {:.4f} '.format(counter,epoch, max_epoch, ret[0].numpy(), ret[1].numpy(), ret[2].numpy(), ret[3].numpy(), ret[4].numpy(), forward_time, batch_time))

            with open('log/train.txt', 'a') as f:
                f.write('train: {} @ epoch:{}/{} loss: {:.4f} reg_loss: {:.4f} cls_loss: {:.4f} cls_pos_loss: {:.4f} cls_neg_loss: {:.4f} forward time: {:.4f} batch time: {:.4f} '.format(counter,epoch, max_epoch, ret[0].numpy(), ret[1].numpy(), ret[2].numpy(), ret[3].numpy(), ret[4].numpy(), forward_time, batch_time))
    model.save_weights(save_model_dir)
                #batches = iterate_data(train_dir)
                #batch = next(batches)
        # save model
        #model.save('/save_model/',save_format='tf')




['aug_000387_2_-0_3711' 'aug_006124_1_404']


TypeError: in converted code:

    /home/hao.wu/Documents/voxelnet_DL/voxelnet_tf2/model/model.py:55 train_step  *
        self.feature(input_feature=voxel_feature,coordinate=voxel_coordinate,training=self.is_train,batch_size=self.batch_size)

    TypeError: tf____call__() got an unexpected keyword argument 'input_feature'


In [ ]:
#tf.config.experimental_run_functions_eagerly(True)
train_summary_writer = tf.summary.create_file_writer('./summaries/train/')
tf.random.set_seed(1)

batch_size=args.single_batch_size
max_epoch = 1

# training only for bn and let it run in training mode but not inference mode
model=RPN3D(cls=cfg.DETECT_OBJ,batch_size=batch_size, alpha=1.0,beta=10.0,training=False)
# save
#model.save_weights(save_model_dir+'save_model_1')
batch_time = time.time()
counter=0
summary_interval = 1
anchors = cal_anchors()
for epoch in range(max_epoch):
    for idx, batch in enumerate(iterate_data(train_dir,shuffle=True,aug=True,batch_size=batch_size,multi_gpu_sum=1,is_testset=False)):

        if idx==2:
            break
        tag = batch[0]
        print(tag)
        # get the data
        voxel_feature,vox_number,voxel_coordinate,pos_equal_one,neg_equal_one,targets,pos_equal_one_for_reg,pos_equal_one_sum,neg_equal_one_sum = get_data(batch)
        counter += 1
        start_time = time.time()

        if counter % summary_interval == 0:
            is_summary = True
        else:
            is_summary = False

        with train_summary_writer.as_default():
            model.train_step(voxel_feature=voxel_feature,vox_number=vox_number,voxel_coordinate=voxel_coordinate,pos_equal_one=pos_equal_one,neg_equal_one=neg_equal_one,targets=targets,pos_equal_one_for_reg=pos_equal_one_for_reg,pos_equal_one_sum=pos_equal_one_sum,neg_equal_one_sum=neg_equal_one_sum,is_summary=is_summary)
        batch_time = time.time() - batch_time
        forward_time = time.time() - start_time

        print('train: {} @ epoch:{}/{} loss: {:.4f} reg_loss: {:.4f} cls_loss: {:.4f} cls_pos_loss: {:.4f} cls_neg_loss: {:.4f} forward time: {:.4f} batch time: {:.4f} '.format(counter,epoch, max_epoch, model.loss.numpy(), model.reg_loss.numpy(), model.cls_loss.numpy(), model.cls_pos_loss.numpy(), model.cls_neg_loss.numpy(), forward_time, batch_time))

        with open('log/train.txt', 'a') as f:
            f.write('train: {} @ epoch:{}/{} loss: {:.4f} reg_loss: {:.4f} cls_loss: {:.4f} cls_pos_loss: {:.4f} cls_neg_loss: {:.4f} forward time: {:.4f} batch time: {:.4f} '.format(counter,epoch, max_epoch, model.loss.numpy(), model.reg_loss.numpy(), model.cls_loss.numpy(), model.cls_pos_loss.numpy(), model.cls_neg_loss.numpy(), forward_time, batch_time))
model.save_weights(save_model_dir)
            #batches = iterate_data(train_dir)
            #batch = next(batches)
    # save model
    #model.save('/save_model/',save_format='tf')

In [131]:
class Net(tf.keras.Model):
    """A simple linear model."""
    def __init__(self):
        super(Net, self).__init__()
        self.epoch = tf.Variable(0, trainable=False)
        self.epoch_add_op = self.epoch.assign(self.epoch + 1)
        #self.epoch = self.epoch+1
        self.l1 = tf.keras.layers.Dense(5)
        self.l2 = tf.keras.layers.Dense(3)
    @tf.function
    def train(self, x):
        self.x=self.l1(x)
        self.x = self.l2(x)
        return self.x

In [132]:
net = Net()
net.train(tf.Variable([[1,2],[3,4]]))
net.save_weights('./save_model/')


ValueError: substring not found

In [129]:
#net = Net()
net.save_weights('./save_model/')


In [ ]:
optimizer = tf.keras.optimizers.Adam(0.1)
loss_object =tf.losses.sparse_categorical_crossentropy
ckpt = tf.train.Checkpoint(step=tf.Variable(0), optimizer=optimizer, model=model)
manager = tf.train.CheckpointManager(ckpt, './save_model/', max_to_keep=3)

def train_one_step(data, label, model, optimizer):
    with tf.GradientTape() as tape:
        pred, end_points= model(data)
        loss = loss_object
        ckpt.step.assign_add(1)
        if int(ckpt.step) % 10 == 0:
        save_path = manager.save()
        print ("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))